# Errores Detectados en el Análisis

### 1. Error de Concepto (El método)

* **El enunciado pide:** "el test de relación que sea necesario hacer".
* **El código hace:** `One-Hot Encoding` + `Correlación`.
* **Por qué está mal:** La correlación (Pearson) es para *número vs número*. Para *categórica vs numérica*, lo estándar en Data Science es un **Test de Hipótesis** de comparación de medias:
    * **ANOVA** (para más de dos grupos).
    * **T-test** (para dos grupos).
* **Explicación visual:** Un ANOVA analiza si la media del precio (`fare`) cambia significativamente dependiendo de la clase (`First`, `Second`, `Third`). La correlación sobre variables *dummy* se considera una aproximación "sucia".

---

### 2. Error de Lógica (El `pvalue`)

* **Línea conflictiva:** ```python
    if abs(corr) >= pvalue:
    ```
* **El argumento:** Se llama `pvalue` y tiene un valor por defecto de `0.05`.
* **El Bug:** Se está comparando una **Correlación** directamente con un **P-Valor**. Son conceptos distintos:
    1.  En estadística, buscamos un **p-value BAJO** ($< 0.05$) para confirmar que algo es significativo.
    2.  El código actual busca una **correlación ALTA** ($> 0.05$).
* **Consecuencia:** Una correlación de `0.1` es estadísticamente bajísima, pero como `0.1 > 0.05`, la función marcará la variable como "importante". 
> [!CAUTION]
> **Resultado:** El script seleccionará casi todas las variables como "buenas" de forma falsa.

In [ ]:
#Codigo Corregido 

import pandas as pd
import numpy as np
from scipy.stats import f_oneway # Importante: Necesario para el test ANOVA

def get_features_cat_regression(df, target_col, pvalue=0.05):
    """
    Devuelve una lista de columnas categóricas que tienen una relación estadísticamente significativa
    con una variable objetivo numérica, utilizando el test ANOVA (Analysis of Variance).

    Argumentos:
    df (pandas.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna numérica que es el target del modelo.
    pvalue (float): Valor de significación estadística (alpha). Por defecto 0.05.
                    Si el p-valor del test es MENOR que este umbral, la variable se selecciona.

    Retorna:
    list: Los nombres de las columnas categóricas relevantes.
          Retorna None si los argumentos de entrada no son válidos.
    """

    # 1. Comprobaciones de seguridad (Input checks)
    if not isinstance(df, pd.DataFrame):
        print("Error: El argumento 'df' debe ser un pandas DataFrame.")
        return None

    if target_col not in df.columns:
        print(f"Error: La columna '{target_col}' no existe en el DataFrame.")
        return None

    # Comprobamos si el target es numérico
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print(f"Error: La columna objetivo '{target_col}' debe ser numérica.")
        return None

    if not isinstance(pvalue, float) or not (0 < pvalue < 1):
        print("Error: 'pvalue' debe ser un float entre 0 y 1.")
        return None

    # 2. Selección de variables categóricas candidatas
    # Incluimos 'object' (strings) y 'category' (pandas categories)
    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

    if not cat_cols:
        print("Aviso: No se encontraron columnas categóricas en el DataFrame.")
        return [] # Devolvemos lista vacía en lugar de None para no romper bucles externos

    columnas_seleccionadas = []

    # 3. Bucle de Análisis (ANOVA)
    for col in cat_cols:
        # Limpiamos nulos solo para este par de variables (features + target)
        temp_df = df[[col, target_col]].dropna()

        # Si tras limpiar nulos no hay datos o solo hay una categoría, saltamos
        if temp_df.empty or temp_df[col].nunique() < 2:
            continue

        # Preparamos los grupos para el ANOVA
        # Ejemplo: grupos = [precios_primera_clase, precios_segunda_clase, ...]
        grupos = []
        for categoria in temp_df[col].unique():
            grupo_valores = temp_df[temp_df[col] == categoria][target_col].values
            grupos.append(grupo_valores)

        # Ejecutamos el test ANOVA (f_oneway)
        # f_oneway devuelve (estadístico, p_valor)
        stat, p_valor_calculado = f_oneway(*grupos)

        # 4. Decisión
        # Si el p-valor es PEQUEÑO (< 0.05), rechazamos la hipótesis nula.
        # Significa que SÍ hay diferencias significativas entre las medias de los grupos.
        if p_valor_calculado < pvalue:
            columnas_seleccionadas.append(col)

    return columnas_seleccionadas

# Análisis de Errores y Corrección Técnica

## 🎓 Explicación Técnica 

Cuando presentes o expliques esto a tu equipo, destaca estos **3 cambios clave**:

1.  **El Import Nuevo:** Hemos añadido `from scipy.stats import f_oneway`. Sin esto no podemos hacer estadística seria.
2.  **La Lógica ANOVA:** En lugar de convertir a números (dummies), agrupamos los datos: *"Toma todos los precios de Primera Clase, todos los de Segunda y todos los de Tercera, y dime si sus medias son distintas"*. Eso hace `f_oneway`.
3.  **La Condición Correcta:** Hemos cambiado `if abs(corr) > pvalue` (que estaba mal) por `if p_valor_calculado < pvalue`.
    * En ciencia, buscamos **p-valores bajos** (menores a 0.05) para confirmar que un hallazgo es real y no suerte.

---

## 🧪 Prueba Final

Si ahora corres tu notebook de test (el que hicimos antes con la "variable basura"), ocurrirá la magia:

* **Variables reales** (`class`, `who`, `embark_town`): Tendrán un p-valor bajísimo (ej. 0.00001) -> **SELECCIONADAS ✅**
* **Variable trampa** (`categoria_basura`): Tendrá un p-valor alto (ej. 0.85) -> **DESCARTADA ❌**